# dependencies
connect to cpu if want to use behaviour cloning


In [1]:
#!git clone http://github.com/HumanCompatibleAI/imitation
#!pip install -e "/content/imitation[dev]"

In [2]:
!pip install gymnasium[mujoco]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.9/953.9 kB 13.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 kB 15.9 MB/s eta 0:00:00


In [3]:
!pip install imitation
!pip install shimmy
!pip install git+https://github.com/Stable-Baselines-Team/stable-baselines3-contrib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.7/181.7 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.9/107.9 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 37.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.4/233.4 kB 23.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823

# environment and expert

In [4]:
import gymnasium as gym
import imitation
import seals

In [5]:
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper
from imitation.algorithms import bc
from imitation.rewards.reward_nets import BasicRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy
import sb3_contrib
from sb3_contrib import TRPO
from imitation.data import rollout
import torch
from torch.optim.adam import Adam
SEED = 42

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [10]:
def make_env_expert_transitions(n_trajs):
    # Create the environment, load expert policy, and sample trajectories
    # from the expert policy and environment.
    global SEED
    env = make_vec_env(
    "seals:seals/HalfCheetah-v1",
    rng=np.random.default_rng(SEED),
    n_envs=n_trajs,
    post_wrappers=[
        lambda env, _: RolloutInfoWrapper(env)
    ],  # needed for computing rollouts later
    )
    expert = load_policy(
        "ppo-huggingface",
        organization="HumanCompatibleAI",
        env_name="seals/HalfCheetah-v1",
        venv=env,
    )

    rollouts = rollout.rollout(
        expert,
        env,
        rollout.make_sample_until(min_timesteps=1000, min_episodes = n_trajs),
        rng=np.random.default_rng(SEED), #The random state to use for sampling trajectories
    )
    transitions = rollout.flatten_trajectories(rollouts)

    return env, expert, transitions


In [7]:
from typing import Callable
from stable_baselines3 import PPO
def linear_schedule(initial_value: float) -> Callable[[float], float]:
    """
    Linear learning rate schedule.

    :param initial_value: Initial learning rate.
    :return: schedule that computes
      current learning rate depending on remaining progress
    """
    def func(progress_remaining: float) -> float:
        """
        Progress will decrease from 1 (beginning) to 0.

        :param progress_remaining:
        :return: current learning rate
        """
        return progress_remaining * initial_value

    return func

In [8]:
def make_BC_trainer(env, expert, transitions):
    global SEED
# 可以调policy network的架构
# Custom actor (pi) and value function (vf) networks
# of two layers of size 32 each with Relu activation function
#policy_kwargs = dict(activation_fn=th.nn.ReLU,
#                     net_arch=[dict(pi=[32, 32], vf=[32, 32])])
    policy_network = MlpPolicy(observation_space = env.observation_space,
                            action_space = env.action_space,
                            lr_schedule = linear_schedule(0.0003034620018780926))

    bc_trainer = bc.BC(
        observation_space=env.observation_space,
        action_space=env.action_space,
        policy = policy_network, # default optimizer is adam
        batch_size = 32,
        optimizer_kwargs = {"lr": 0.0003034620018780926},
        demonstrations=transitions,
        rng=np.random.default_rng(SEED),
    )
    return bc_trainer

# BC - 1 trajectory

In [99]:
env_1, expert_1, transitions_1 = make_env_expert_transitions(1)

In [100]:
bc_trainer_1 = make_BC_trainer(env_1, expert_1, transitions_1)

In [101]:
expert_reward, _ = evaluate_policy(
    expert_1, env_1, 20, return_episode_rewards=True
)
print(
    "expert rewards:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards: 1654.6617238499998 +/- 46.922795088115606


In [102]:
env_1.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    bc_trainer_1.policy, env_1, 20, return_episode_rewards=True
)

In [103]:
env_1.seed(SEED)
bc_trainer_1.train(n_epochs = 300, log_interval = 500)

0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.00851 |
|    entropy        | 8.51     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 165      |
|    loss           | 7.19     |
|    neglogp        | 7.2      |
|    prob_true_act  | 0.0008   |
|    samples_so_far | 32       |
--------------------------------


20batch [00:00, 195.91batch/s]
59batch [00:00, 184.46batch/s]
78batch [00:00, 180.64batch/s]
116batch [00:00, 179.93batch/s]
155batch [00:00, 181.94batch/s]
174batch [00:00, 178.91batch/s]
213batch [00:01, 182.01batch/s]
232batch [00:01, 178.77batch/s]
270batch [00:01, 180.71batch/s]
309batch [00:01, 183.61batch/s]
328batch [00:01, 180.48batch/s]
368batch [00:02, 182.86batch/s]
387batch [00:02, 179.53batch/s]
424batch [00:02, 178.49batch/s]
461batch [00:02, 180.16batch/s]
480batch [00:02, 179.26batch/s]
498batch [00:02, 178.49batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 500      |
|    ent_loss       | -0.00749 |
|    entropy        | 7.49     |
|    epoch          | 16       |
|    l2_loss        | 0        |
|    l2_norm        | 171      |
|    loss           | 4.94     |
|    neglogp        | 4.95     |
|    prob_true_act  | 0.00796  |
|    samples_so_far | 16032    |
--------------------------------


517batch [00:02, 179.27batch/s]
553batch [00:03, 176.27batch/s]
571batch [00:03, 176.58batch/s]
609batch [00:03, 179.86batch/s]
647batch [00:03, 180.77batch/s]
666batch [00:03, 179.08batch/s]
704batch [00:03, 183.01batch/s]
743batch [00:04, 183.81batch/s]
763batch [00:04, 186.59batch/s]
802batch [00:04, 187.43batch/s]
821batch [00:04, 182.99batch/s]
859batch [00:04, 179.89batch/s]
898batch [00:04, 182.21batch/s]
917batch [00:05, 182.38batch/s]
956batch [00:05, 184.34batch/s]
975batch [00:05, 184.11batch/s]
994batch [00:05, 182.10batch/s]

...

8020batch [00:45, 174.39batch/s]
8058batch [00:46, 180.26batch/s]
8077batch [00:46, 175.10batch/s]
8113batch [00:46, 172.13batch/s]
8152batch [00:46, 179.91batch/s]
8171batch [00:46, 177.21batch/s]
8208batch [00:47, 176.99batch/s]
8245batch [00:47, 177.22batch/s]
8263batch [00:47, 173.37batch/s]
8300batch [00:47, 172.70batch/s]
8337batch [00:47, 173.48batch/s]
8355batch [00:47, 169.74batch/s]
8392batch [00:48, 171.50batch/s]
8430batch [00:48, 177.83batch/s]
8448batch [00:48, 169.27batch/s]
8484batch [00:48, 172.38batch/s]
Epoch 273 of 300                

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 8500     |
|    ent_loss       | 0.00378  |
|    entropy        | -3.78    |
|    epoch          | 274      |
|    l2_loss        | 0        |
|    l2_norm        | 248      |
|    loss           | -4.12    |
|    neglogp        | -4.12    |
|    prob_true_act  | 194      |
|    samples_so_far | 272032   |
--------------------------------


8520batch [00:48, 171.03batch/s]
8538batch [00:48, 168.58batch/s]
8575batch [00:49, 173.57batch/s]
8613batch [00:49, 177.93batch/s]
8631batch [00:49, 172.69batch/s]
8669batch [00:49, 177.77batch/s]
8707batch [00:49, 183.22batch/s]
8726batch [00:50, 175.51batch/s]
8763batch [00:50, 179.46batch/s]
8800batch [00:50, 174.38batch/s]
8818batch [00:50, 175.11batch/s]
8857batch [00:50, 181.08batch/s]
8894batch [00:50, 177.74batch/s]
8912batch [00:51, 174.49batch/s]
8949batch [00:51, 177.94batch/s]
8986batch [00:51, 179.33batch/s]
Epoch 289 of 300                

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 9000     |
|    ent_loss       | 0.00404  |
|    entropy        | -4.04    |
|    epoch          | 290      |
|    l2_loss        | 0        |
|    l2_norm        | 252      |
|    loss           | -4.49    |
|    neglogp        | -4.49    |
|    prob_true_act  | 306      |
|    samples_so_far | 288032   |
--------------------------------


9004batch [00:51, 178.24batch/s]
9043batch [00:51, 183.96batch/s]
9081batch [00:52, 177.36batch/s]
9100batch [00:52, 180.38batch/s]
9140batch [00:52, 186.70batch/s]
9159batch [00:52, 184.39batch/s]
9197batch [00:52, 182.51batch/s]
9234batch [00:52, 163.68batch/s]
9252batch [00:53, 166.22batch/s]
9289batch [00:53, 172.70batch/s]
9300batch [00:53, 174.56batch/s]


In [104]:
env_1.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    bc_trainer_1.policy, env_1, 20, return_episode_rewards=True
)

In [105]:
print(
    "learner rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)

learner rewards before training: -0.26874590000000004 +/- 0.6310643279053174


In [106]:
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

learner rewards after training: 1183.3321252 +/- 382.6355882662263


In [110]:
import torch
torch.save(bc_trainer_1.policy, "BC_MlpPolicy_HalfCheetah_1.pth.tar")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# BC - 4 trajectory

In [91]:
env_4, expert_4, transitions_4 = make_env_expert_transitions(4)

In [92]:
bc_trainer_4 = make_BC_trainer(env_4, expert_4, transitions_4)

In [93]:
expert_reward, _ = evaluate_policy(
    expert_4, env_4, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 1665.53076195 +/- 28.040166619644488


In [94]:
env_4.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    bc_trainer_4.policy, env_4, 20, return_episode_rewards=True
)

In [95]:
env_4.seed(SEED)
bc_trainer_4.train(n_epochs = 300, log_interval = 500)

0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.00851 |
|    entropy        | 8.51     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 165      |
|    loss           | 7.21     |
|    neglogp        | 7.22     |
|    prob_true_act  | 0.000782 |
|    samples_so_far | 32       |
--------------------------------


120batch [00:00, 191.14batch/s]
235batch [00:01, 185.09batch/s]
361batch [00:02, 171.50batch/s]
491batch [00:02, 180.64batch/s]
Epoch 3 of 300                 

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 500      |
|    ent_loss       | -0.0075  |
|    entropy        | 7.5      |
|    epoch          | 4        |
|    l2_loss        | 0        |
|    l2_norm        | 170      |
|    loss           | 4.87     |
|    neglogp        | 4.88     |
|    prob_true_act  | 0.00822  |
|    samples_so_far | 16032    |
--------------------------------


608batch [00:03, 189.19batch/s]
743batch [00:04, 187.03batch/s]
861batch [00:04, 191.39batch/s]
983batch [00:05, 192.06batch/s]
Epoch 7 of 300                 

...

36110batch [03:19, 190.08batch/s]
36247batch [03:19, 188.52batch/s]
36365batch [03:20, 184.94batch/s]
36500batch [03:21, 190.82batch/s]
Epoch 291 of 300                 

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 36500    |
|    ent_loss       | 0.00413  |
|    entropy        | -4.13    |
|    epoch          | 292      |
|    l2_loss        | 0        |
|    l2_norm        | 309      |
|    loss           | -4.23    |
|    neglogp        | -4.24    |
|    prob_true_act  | 219      |
|    samples_so_far | 1168032  |
--------------------------------


36617batch [03:21, 186.44batch/s]
36736batch [03:22, 185.54batch/s]
36857batch [03:23, 196.69batch/s]
36994batch [03:23, 180.58batch/s]
Epoch 295 of 300                 

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 37000    |
|    ent_loss       | 0.00414  |
|    entropy        | -4.14    |
|    epoch          | 296      |
|    l2_loss        | 0        |
|    l2_norm        | 310      |
|    loss           | -5.34    |
|    neglogp        | -5.34    |
|    prob_true_act  | 372      |
|    samples_so_far | 1184032  |
--------------------------------


37109batch [03:24, 180.06batch/s]
37242batch [03:25, 182.17batch/s]
37362batch [03:25, 156.49batch/s]
37497batch [03:26, 161.18batch/s]
37500batch [03:26, 181.33batch/s]


In [96]:
env_4.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    bc_trainer_4.policy, env_4, 20, return_episode_rewards=True
)

In [97]:
print(
    "learner rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)

learner rewards before training: -0.030640249999999987 +/- 0.6043992626636696


In [98]:
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

learner rewards after training: 1401.4868689 +/- 334.43244982744176


In [108]:
import torch
torch.save(bc_trainer_4.policy, "BC_MlpPolicy_HalfCheetah_4.pth.tar")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# BC - 10 trajectory

In [67]:
env_10, expert_10, transitions_10 = make_env_expert_transitions(10)

In [68]:
bc_trainer_10 = make_BC_trainer(env_10, expert_10, transitions_10)

In [69]:
expert_reward, _ = evaluate_policy(
    expert_10, env_10, 20, return_episode_rewards=True
)
print(
    "expert rewards after training:",
    np.mean(expert_reward),
    "+/-",
    np.std(expert_reward),
)

expert rewards after training: 1675.0586300999998 +/- 44.32240034603762


In [70]:
env_10.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    bc_trainer_10.policy, env_10, 20, return_episode_rewards=True
)

In [71]:
env_10.seed(SEED)
bc_trainer_10.train(n_epochs = 200, log_interval = 500)

0batch [00:00, ?batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 0        |
|    ent_loss       | -0.00851 |
|    entropy        | 8.51     |
|    epoch          | 0        |
|    l2_loss        | 0        |
|    l2_norm        | 165      |
|    loss           | 7.25     |
|    neglogp        | 7.25     |
|    prob_true_act  | 0.000768 |
|    samples_so_far | 32       |
--------------------------------


298batch [00:01, 191.74batch/s]
499batch [00:02, 150.75batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 500      |
|    ent_loss       | -0.00749 |
|    entropy        | 7.49     |
|    epoch          | 1        |
|    l2_loss        | 0        |
|    l2_norm        | 170      |
|    loss           | 4.91     |
|    neglogp        | 4.92     |
|    prob_true_act  | 0.0078   |
|    samples_so_far | 16032    |
--------------------------------


614batch [00:03, 158.64batch/s]
918batch [00:05, 202.63batch/s]
1000batch [00:05, 188.57batch/s]

...

61134batch [05:33, 183.78batch/s]
61456batch [05:35, 186.88batch/s]
61495batch [05:35, 183.84batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 61500    |
|    ent_loss       | 0.0041   |
|    entropy        | -4.1     |
|    epoch          | 197      |
|    l2_loss        | 0        |
|    l2_norm        | 330      |
|    loss           | -4.09    |
|    neglogp        | -4.1     |
|    prob_true_act  | 313      |
|    samples_so_far | 1968032  |
--------------------------------


61776batch [05:37, 197.20batch/s]
61998batch [05:38, 196.98batch/s]

--------------------------------
| batch_size        | 32       |
| bc/               |          |
|    batch          | 62000    |
|    ent_loss       | 0.0041   |
|    entropy        | -4.1     |
|    epoch          | 198      |
|    l2_loss        | 0        |
|    l2_norm        | 331      |
|    loss           | -4.69    |
|    neglogp        | -4.7     |
|    prob_true_act  | 332      |
|    samples_so_far | 1984032  |
--------------------------------


62078batch [05:38, 192.57batch/s]
62391batch [05:40, 175.28batch/s]
62400batch [05:40, 183.25batch/s]


In [72]:
env_10.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    bc_trainer_10.policy, env_10, 20, return_episode_rewards=True
)

In [73]:
print(
    "learner rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)

learner rewards before training: -0.3925922 +/- 0.527445616683616


In [74]:
print(
    "learner rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

learner rewards after training: 1539.0253395 +/- 291.0754722627695


In [111]:
import torch
torch.save(bc_trainer_10.policy, "BC_MlpPolicy_HalfCheetah_10.pth.tar")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
